# 1. Собрать данные по дневным ценам активов (акций) и дневным объемам продаж на заданном фондовом рынке за указанный период. Добавить (если нет) данные по индексу рынка.

In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

**Скачал список компаний Франкфуртской биржи**

In [2]:
companies_list_file_name = 'list.xlsx'

In [ ]:
companies_list = pd.read_excel(companies_list_file_name)
companies_list

In [ ]:
start_date = '2019-01-02'
end_date = '2020-01-01'

def download_ticker(ticker, start=start_date, end=end_date):
    return yf.download(ticker, start, end)

**Индекс DAX представляет собой главный индекс Франфуртской биржи.  Индекс вычисляется как среднее взвешенное по капитализации значение цен акций крупнейших акционерных компаний Германии (при этом капитализация рассчитывается лишь на основе акций, находящихся в свободном обращении). Индекс учитывает также полученные в виде дивидендов доходы по акциям, предполагая, что дивиденды реинвестируются в акцию, по которой был получен данный дивиденд. Таким образом, индекс отражает суммарный доход по капиталу.**

In [ ]:
DAX_index = download_ticker('^GDAXI')
DAX_index.drop(['Open', 'High', 'Low', 'Close'], axis=1, inplace=True)
DAX_index

**Выкачал тикеры, удалил из датафрейма компании которые не скачались. Перекачивать не нужно, сохранено в бинарник.**

In [ ]:
if False:
    tickers = [None] # None для нормальной адресации - в фрейме с компаниями индексация начинается с единицы
    failed = []
    for index, row in companies_list.iterrows():
        ticker = str(row['Ticker'])
        ticker_data = download_ticker(ticker)
        if ticker_data.shape[0] == 0:
            failed.append(index)
            continue
        tickers.append(ticker_data)
    companies_list = companies_list.drop(failed)
    companies_list

**Следующая ячейка загружает обьекты из файлов в память**

In [ ]:
companies_list_csv = 'companies_list.csv'
companies_list = pd.read_csv(companies_list_csv, index_col='index')
companies_list

In [ ]:
from os import listdir
from os.path import isfile, join
tickers_folder = 'tickers'
onlyfiles = [f for f in listdir(tickers_folder) if isfile(join(tickers_folder, f))]
tickers = []
for i in range(len(onlyfiles)):
    path = tickers_folder + '/' + str(i)
    tickers.append(pd.read_csv(path, index_col='Date'))
tickers[0]

In [ ]:
len(tickers)

# 2. Преобразовать данные по ценам в данные по доходностям (используйте логарифмическую доходность). Вычислите оценки ожидаемых доходностей и стандартных отклонений и постройте «карту» активов в системе координат (σ, E).

**По взвешенной цене закрытия считаю логарифмические доходности за периоды.**

In [ ]:
count = 0
for ticker in tickers:
    log_returns = []
    close_price = ticker['Adj Close']
    log_returns.append(None)
    for i in range(1, len(close_price)):
        log_return = (close_price[i] - close_price[i - 1]) / close_price[i - 1]
        log_returns.append(log_return)
    ticker['LogReturn'] = log_returns
    count += 1

In [ ]:
tickers[0]

In [ ]:
companies_list.reset_index(inplace=True)
companies_list

**Для каждого актива высчитываю логарифмическую доходность за год (по сумме доходностей за отдельные периоды), а также оценку доходности как среднее от доходностей за отдельные периоды.**

In [ ]:
companies_list['Annual LogReturn'] = [tickers[i]['LogReturn'].sum() for i, row in companies_list.iterrows()]
companies_list['E'] = [tickers[i]['LogReturn'].mean() for i, row in companies_list.iterrows()]

**Риск - стандартное отклонение доходности**

In [ ]:
companies_list['Risk'] = [tickers[i]['LogReturn'].std() for i, row in companies_list.iterrows()]

In [ ]:
plt.figure(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(companies_list['Risk'], companies_list['E'], label='Актив')
ax.legend()
plt.xlabel('σ (Риск)')
plt.ylabel('E (Лог. доходность)')

**В некоторых слишком мало наблюдений, удалил все где меньше 100 периодов.**

In [ ]:
little_observations = []
for i in range(len(tickers)):
    if len(tickers[i]) < 100:
        little_observations.append(i)

In [ ]:
little_observations

In [ ]:
companies_list = companies_list.reset_index(drop=True)

**Итоговые данные с которыми можно работать:**

In [ ]:
companies_list

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(companies_list['Risk'], companies_list['E'], label='Актив')
ax.legend()
plt.xlabel('σ (Риск)')
plt.ylabel('E (Лог. доходность)')

In [ ]:
for i, row in companies_list.iterrows():
    if row['Risk'] > 10:
        companies_list.drop(i, inplace=True)

In [ ]:
plt.figure(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(companies_list['Risk'], companies_list['E'], label='Актив')
ax.legend()
plt.xlabel('σ (Риск)')
plt.ylabel('E (Лог. доходность)')

In [ ]:
for i, row in companies_list.iterrows():
    if row['Risk'] > 1:
        companies_list.drop(i, inplace=True)

In [ ]:
plt.figure(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(companies_list['Risk'], companies_list['E'], label='Актив')
ax.legend()
plt.xlabel('σ (Риск)')
plt.ylabel('E (Лог. доходность)')

In [ ]:
for i, row in companies_list.iterrows():
    if row['Risk'] > 0.1:
        companies_list.drop(i, inplace=True)

In [ ]:
plt.figure(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(companies_list['Risk'], companies_list['E'], label='Актив')
ax.legend()
plt.xlabel('σ (Риск)')
plt.ylabel('E (Лог. доходность)')

Даже несмотря на то, что получилось собрать не очень много данных о компаниях, карта активов дает довольно четкий рисунок. Можно увидеть, что активы имеют тенденцию располагаться примерно в одной области графика.

# 3. Рассмотрите портфель с равными долями капитала и отметьте его на карте активов в системе координат (σ, E). Дайте характеристику этому портфелю.

In [ ]:
balanced_E = companies_list['E'].mean()
balanced_Risk = companies_list['E'].std()
print('Сбалансированный портфель:\nE =', balanced_E, '\nσ =', balanced_Risk)

In [ ]:
plt.figure(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(companies_list['Risk'], companies_list['E'], label='Актив')
ax.scatter([balanced_Risk], [balanced_E], marker='x', color='red', s=[250], label='Равномерный портфель')
ax.legend()
plt.xlabel('σ (Риск)')
plt.ylabel('E (Лог. доходность)')
plt.show()

На карте видно, что равномерный портфель имеет небольшой риск и небольшую доходность относительно других активов. При этом доходность положительная.\
Возможно, портфель является Парето-оптимальным, так как находится на границе обозримой области портфелей.

# 4. Рассмотрите индекс рынка и отметьте его на карте активов в системе координат (σ,E). Дайте характеристику индексу рынка.

In [ ]:
log_returns = []
close_price = DAX_index['Adj Close']
log_returns.append(None)
for i in range(1, len(close_price)):
    log_return = (close_price[i] - close_price[i - 1]) / close_price[i - 1]
    log_returns.append(log_return)
DAX_index['LogReturn'] = log_returns
DAX_index

In [ ]:
dax_annual_log_return = DAX_index['LogReturn'].sum()
dax_E = DAX_index['LogReturn'].mean()
dax_Risk = DAX_index['LogReturn'].std()
print('Годовая логарифмическая доходность рынка по индексу DAX =', dax_annual_log_return)
print('Оценка логарифмическая доходности рынка по индексу DAX =', dax_E)
print('Оценка рынка по индексу DAX =', dax_Risk)

In [ ]:
plt.figure(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(companies_list['Risk'], companies_list['E'], label='Актив')
ax.scatter([balanced_Risk], [balanced_E], marker='x', color='red', s=[250], label='Равномерный портфель')
ax.scatter([dax_Risk], [dax_E], marker='+', color='red', s=[250], label='Индекс рынка')
ax.legend()
plt.xlabel('σ (Риск)')
plt.ylabel('E (Лог. доходность)')
plt.show()

Индекс рынка имеет риск немного больше, чем равномерный портфель, при этом его доходность почти не отличается от равномерного портфеля.\
Можно предположить, что это означает, что рынок находится в стабильном состоянии и медленно растет.\
Индекс рынка не является Парето-оптимальным.

# 5. Задайте уровень риска и оцените VaR (Value at Risk = Сумма под Риском) для активов вашего рынка.

Для нахождения суммы под риском,были выбраны два показателя доверия гамма: 0.9 и 0.95. Сумма под риском - квантиль уровня гамма, где гамма — это наименьшая величина потерь, такая, что убыток превысит это значение с вероятностью не большей (1 - гамма)

Получаем точку, которая соответствовует требуемому уровню доверительной вероятности

In [ ]:
import copy
import numpy as np

Var1_np = []
Var2_np = []

for index in list(companies_list.index):

    simpleReturns = tickers[index]['LogReturn'].dropna()

    if (len(simpleReturns) != 0):
      Var1_np.append(-np.quantile(simpleReturns, 1-0.9))
      Var2_np.append(-np.quantile(simpleReturns, 1-0.95))

**Для $\gamma$ = 0.9**

In [ ]:
import matplotlib.pyplot as plt
plt.hist(Var1_np, bins=20)
plt.show()

plt.hist(Var1_np, bins=20)
plt.show()

In [ ]:
best_VaR_1 = min(Var1_np)
pd.set_option("display.max_columns", None)
best_VaR_1_idx = Var1_np.index(best_VaR_1)
print(companies_list.loc[[65]]) 

In [ ]:
plt.figure(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(companies_list['Risk'], companies_list['E'], label='Актив')
ax.scatter([balanced_Risk], [balanced_E], marker='x', color='red', s=[250], label='Равномерный портфель')
ax.scatter([0.009964], [0.0012], marker='+', color='green', s=[250], label='Наиболее предпочтительный актив') 
ax.scatter([dax_Risk], [dax_E], marker='+', color='red', s=[250], label='Индекс рынка')
ax.legend()
plt.xlabel('σ (Риск)')
plt.ylabel('E (Лог. доходность)')
plt.show()

**Для $\gamma$ = 0.95**

In [ ]:
import matplotlib.pyplot as plt
plt.hist(Var2_np, bins=20)
plt.show()

plt.hist(Var2_np, bins=20)
plt.show()

In [ ]:
best_VaR_2 = min(Var2_np)
pd.set_option("display.max_columns", None)
best_VaR_2_idx = Var2_np.index(best_VaR_2)
print(companies_list.loc[[61]]) 

In [ ]:
plt.figure(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(companies_list['Risk'], companies_list['E'], label='Актив')
ax.scatter([balanced_Risk], [balanced_E], marker='x', color='red', s=[250], label='Равномерный портфель')
ax.scatter([0.025726  ], [0.000903], marker='+', color='green', s=[250], label='Наиболее предпочтительный актив') 
ax.scatter([dax_Risk], [dax_E], marker='+', color='red', s=[250], label='Индекс рынка')
ax.legend()
plt.xlabel('σ (Риск)')
plt.ylabel('E (Лог. доходность)')
plt.show()

**VAR сбалансированного**

In [ ]:
print(" Потери не превысят ",-np.quantile(companies_list['Annual LogReturn'], 1-0.9),  "с 90% уверенностью")
print(" Потери не превысят ",-np.quantile(companies_list['Annual LogReturn'], 1-0.95),  "с 95% уверенностью")

**VAR индекса рынка**

In [ ]:
log_returns = []
close_price = DAX_index['Adj Close']
log_returns.append(None)
for i in range(1, len(close_price)):
    log_return = (close_price[i] - close_price[i - 1]) / close_price[i - 1]
    log_returns.append(log_return)
print(" Потери не превысят ",-np.quantile(list(filter(None, log_returns)), 1-0.9),  "с 90% уверенностью")
print(" Потери не превысят ",-np.quantile(list(filter(None, log_returns)), 1-0.95),  "с 95% уверенностью")

По характеристике VaR индекс рынка является приоритетнее сбалансированного портфеля

# 6. Выберите несколько интересных (значимых) активов рынка. Можно ли считать наблюдаемые доходности (объемы продаж) конкретного актива повторной выборкой из некоторого распределения (белый шум)?

In [ ]:
dax_companies = [5, 7, 34, 41, 44, 51]
for c in dax_companies:
    print(companies_list.loc[c]['Company'])

Указанные выше компании были сочтены интересными, так как они входят в список 30 компаний, по которым рассчитывается индекс Франкфуртского фондового рынка **DAX**.

In [ ]:
for i in dax_companies:
    print(len(tickers[i]))

По всем шести компаниям есть достаточное число наблюдений для проверки гипотезы.

### Гипотеза о случайности
$X = (X_1, X_2 ... X_n)$ - выборка.\
$H_0$ - гипотеза случайности. Если она верна, то случайные величины в выборке независимы и не являются повторной выборкой из распределения.\
$H_1$ - гипотеза случайности отвергается, данные являются повторной выборкой

#### Критерий инверсий
Говорят, что компоненты $X_i, X_j$ образуют инверсию, если $i<j$ и $X_i > X_j$
Общее число инверсий можно назвать рассматривать как статистику $T_n$ для выборки.\
Интервал значений статистики: $0 <= T_n <= \frac{n(n-1)}{2}$\
Чем ближе значение к границам интервала, тем больше **упорядоченность** данных. Гипотеза случайности отклоняется если данные упорядоченны, не хаотичны.

Матожидание и дисперсия статистики $T_n$ По Ивченко и Медведеву:\
$E(T_n)=\frac{n(n-1)}{4}$\
$D(T_n)=\frac{n(n-1)(2n+5)}{72}$

Гипотеза не будет отвергнута, если нормализованная статистика $T_n$ не будет превышать процентную точку для заданного уровня доверия $\alpha$.
Нормализованная статистика $T_n$ вычисляется по формуле: $\frac{T_n - E(T_n)}{\sqrt{D(T_n)}}$ \
Распространенный уровень доверия $\alpha$ - 0.05

In [ ]:
def count_inversions(arr):
    inv_count = 0
    for i in range(len(arr)-1):
        #print(i, end=', ')
        for j in arr[i+1:]:
            #print(j, end=' ')
            if arr[i] > j:
                inv_count += 1
        #print()
    return inv_count

In [ ]:
import scipy.stats as stats

**Проверяем на случайность доходности**

In [ ]:
for i in dax_companies:
    alpha = 0.05
    returns = tickers[i]['LogReturn']
    Tn = count_inversions(returns)
    n = len(returns)
    E_Tn = (n*(n-1))/4
    D_Tn = (n*(n-1)*(2*n+5))/72
    normalized_Tn = (Tn - E_Tn)/(D_Tn**(1/2))
    p_value = round(stats.norm.sf(abs(normalized_Tn))*2, 2)
    if abs(normalized_Tn) >= stats.norm.ppf(1 - alpha/2):
        print('Гипотеза для компании', companies_list.loc[i]['Company'], 'отвергается, p_value', p_value)
    else:
        print('Гипотеза для компании', companies_list.loc[i]['Company'], 'принимается, p_value', p_value)

Делаем вывод, что данные по доходностям по всем компаниям, кроме "Linde", являются случайными.

**Проверяем на случайность объемы продаж**

In [ ]:
for i in dax_companies:
    alpha = 0.05
    returns = tickers[i]['Volume']
    Tn = count_inversions(returns)
    n = len(returns)
    E_Tn = (n*(n-1))/4
    D_Tn = (n*(n-1)*(2*n+5))/72
    normalized_Tn = (Tn - E_Tn)/(D_Tn**(1/2))
    p_value = round(stats.norm.sf(abs(normalized_Tn))*2, 2)
    if abs(normalized_Tn) >= stats.norm.ppf(1 - alpha/2):
        print('Гипотеза для компании', companies_list.loc[i]['Company'], 'отвергается, p_value', p_value)
    else:
        print('Гипотеза для компании', companies_list.loc[i]['Company'], 'принимается, p_value', p_value)

Делаем вывод, что данные по объемам продаж являются случайными только для компаний "HEIDELBERGCEMENT" и "MERCK".

# 7. Выберите несколько интересных (значимых) активов рынка из разных производственных секторов. В предположении, что наблюдаемые доходности (объемы продаж) являются повторной выборкой из некоторого распределения исследовать (выборочно) распределения доходностей и объемов продаж выбранных активов.

Для отбора значимых активов используется кэффициент sharp-ratio.

In [ ]:
companies_list['SR'] = (companies_list['E'] - companies_list['E'][40])/ companies_list['Risk']

In [ ]:
companies_list.sort_values(by='SR')[-20:]

Получаем список активов, наиболее интересных к рассмотрению

In [ ]:
print(companies_list.loc[[58, 41, 15, 39, 34]])

Предположим, что доходности выбранных активов имеют нормальное распределение, строим гистограммы доходностей

In [ ]:
import seaborn as sns
stocks_inds = [58, 41, 15, 39, 34]
for ind in stocks_inds:
  plt.grid()
  sns.distplot(tickers[ind]['LogReturn'], bins=10)
  plt.title(companies_list.loc[ind]['Company'])
  plt.show()

По гистограммам можно видеть, что скорее всего нормальное распределение доходностей имеют компании  DELTICOM и HEIDELBERGCEMENT.

In [ ]:
import seaborn as sns
stocks_inds = [58, 41, 15, 39, 34]
for ind in stocks_inds:
  plt.grid()
  sns.distplot(tickers[ind]['Volume'], bins=10)
  plt.title(companies_list.loc[ind]['Company'])
  plt.show()

По гистограммам можно видеть, что скорее всего нормального распределение продаж не имеет ни одна компания из выбранных.

Построим тесты для проверки гипотезы о нормальности распределения доходности с уровнем значимости α = 0.05, для проверки полученных диаграмм. Используем тесты Шапиро, ДАгостино, Андерсона


In [ ]:
from scipy.stats import shapiro, normaltest, anderson
tests = {"Shapiro" : shapiro,
 "DAgostino" : normaltest,
 "Anderson" : anderson, }
def test_gipothesys(label, alfa=0.05):
  for test_name, test_f in tests.items():
    result = test_f(label)
    if test_name == 'Anderson':
      statistic = result[0]
      answer = 'отклоняется' if statistic > result[1][2] else 'не отклоняется'
      print("\t Гипотеза {} {}, статистика={:3f}".format(answer, test_name, statistic))
    else:
      p_value = result[1]
      answer = 'не отклоняется' if p_value > alfa else 'отклоняется'
      print("\t Гипотеза {} {}, p-value={:3f}".format(answer, test_name, p_value))


In [ ]:
stocks_names = [58, 41, 15, 39, 34]
stocks = ['SINGULUS TECHNOL. EO 1,Industrial', 'LINDE PLC EO 0,001,Chemicals', 'DELTICOM AG  NA O.N.,Retail', 'LEG IMMOBILIEN SE,Financial Services', 'HEIDELBERGCEMENT AG O.N.,Construction']
print('Доходности:\n\n')
k = 0
for label in stocks_names:
 print('\n', stocks[k], ':\n')
 k += 1
 test_gipothesys(tickers[label]['LogReturn'].dropna())

Результат проверки гипотез статистическими методами совпадает с выводами, полученными при помощи визуального анализа гистограмм - для DELTICOM и HEIDELBERGCEMENT гипотеза о нормальности распределения доходностей не отвергается.

In [ ]:
stocks_names = [58, 41, 15, 39, 34]
stocks = ['SINGULUS TECHNOL. EO 1,Industrial', 'LINDE PLC EO 0,001,Chemicals', 'DELTICOM AG  NA O.N.,Retail', 'LEG IMMOBILIEN SE,Financial Services', 'HEIDELBERGCEMENT AG O.N.,Construction']
print('Объем продаж:\n\n')
k = 0
for label in stocks_names:
 print('\n', stocks[k], ':\n')
 k += 1
 test_gipothesys(tickers[label]['Volume'].dropna())

Результат проверки гипотез статистическими методами совпадает с выводами, полученными при помощи визуального анализа гистограмм - ни одна компания не имеет нормального распределения объема продаж.

Как было показано выше, для двух активов: DELTICOM и HEIDELBERGCEMENT гипотеза о нормальности распределения доходностей не отвергается.\
Рассмотрим эти два актива.

In [ ]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

def plot_stock(y):
 y1 = tickers[15]['LogReturn'].dropna()
 y2 = tickers[34]['LogReturn'].dropna()
 x1 = tickers[15]['LogReturn'].dropna().keys()

 figure(figsize=(20, 6))
 plt.grid()
 plt.plot(x1, y1, marker='', color='skyblue', linewidth=2)
 plt.plot(x1, y2, marker='', color='green', linewidth=2)
 plt.xlabel('Time')
 plt.legend()


plot_stock('LogReturn')

In [ ]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

def plot_stock(y):
 y1 = tickers[15]['Volume'].dropna()
 y2 = tickers[34]['Volume'].dropna()
 x1 = tickers[15]['Volume'].dropna().keys()
 x1 = tickers[15]['Volume'].dropna().keys()

 figure(figsize=(20, 6))
 plt.grid()
 plt.plot(x1, y1, marker='', color='skyblue', linewidth=2)
 plt.plot(x1, y2, marker='', color='green', linewidth=2)
 plt.xlabel('Volume')
 plt.legend()


plot_stock('Volume')

По графику можно заметить, что тренды доходностей и объёмов продаж у обеих компаний не совпадают, не смотря одинаковое распределение. Такие результаты мы можем наблюдать по причине того, что активы взяты из разных секторов. Такие активы могут не иметь общих внешних факторов.

# 8. Исследовать зависимости (выборочно) между доходностями различных активов с помощью техники анализа зависимостей.


In [ ]:
!pip install prettytable
from prettytable import PrettyTable
def some_print(df):
 x = PrettyTable()
 x.field_names = df.columns.tolist()
 for row in df.values:
  x.add_row(row)
 print(x)
 return x


Найдём корреляцию между случайными величинами для того, чтобы определить есть ли между ними зависмость

Анализ зависимости доходностей у активов из одного производственного сектора:

In [ ]:
import numpy as np

stock2 = tickers[39]['LogReturn'].dropna()
stock1 = tickers[73]['LogReturn'].dropna()
if (len(stock1) > len(stock2)):
  stock1 = stock1[:len(stock2)]
if (len(stock2) > len(stock1)):
  stock2 = stock2[:len(stock1)]

keys = ['LEG IMMOBILIEN SE,Financial Services', 'GSW IMMOBILIEN AG,Financial Services']
if stock1.shape != stock2.shape:
  new_size = stock1.shape[0]
  stock1 = stock1[:new_size]
  stock2 = stock2[:new_size]
concat_pd = pd.concat([stock1, stock2], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(stock1,stock2)[0][1]

plt.grid()
plt.scatter(stock1, stock2,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('LEG IMMOBILIEN SE,Financial Services'+' and '+'GSW IMMOBILIEN AG,Financial Services', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между LEG IMMOBILIEN SE,Financial Services  и GSW IMMOBILIEN AG,Financial Services  равен {round(corr_value, 3)}')

Коэфициент корреляции 0.613 свидетельствует о линейной зависимости. что подтверждается графиком.
Активы LEG IMMOBILIEN SE,Financial Services и GSW IMMOBILIEN AG,Financial Services из финансового производственного сектора зависимы между собой.

In [ ]:
import numpy as np

stock2 = tickers[41]['LogReturn'].dropna()
stock1 = tickers[79]['LogReturn'].dropna()
if (len(stock1) > len(stock2)):
  stock1 = stock1[:len(stock2)]
if (len(stock2) > len(stock1)):
  stock2 = stock2[:len(stock1)]

keys = ['LINDE PLC EO 0,001,Chemicals,Industrial', 'SIMONA AG O.N.,Chemicals']
if stock1.shape != stock2.shape:
  new_size = stock1.shape[0]
  stock1 = stock1[:new_size]
  stock2 = stock2[:new_size]
concat_pd = pd.concat([stock1, stock2], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(stock1,stock2)[0][1]

plt.grid()
plt.scatter(stock1, stock2,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('LINDE PLC EO 0,001,Chemicals,Industrial'+' and '+'SIMONA AG O.N.,Chemicals', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между 4LINDE PLC EO 0,001,Chemicals,Industrial  и SIMONA AG O.N.,Chemicals  равен {round(corr_value, 3)}')

Коэфициент корреляции 0.414 > 0, что говорит о прямой линейной зависимости. что подтверждается графиком.
Активы LINDE PLC EO 0,001,Chemicals,Industrial и SIMONA AG O.N.,Chemicals из сектора производства химикатов зависимы между собой.

In [ ]:
import numpy as np

stock2 = tickers[20]['LogReturn'].dropna()
stock1 = tickers[46]['LogReturn'].dropna()
if (len(stock1) > len(stock2)):
  stock1 = stock1[:len(stock2)]
if (len(stock2) > len(stock1)):
  stock2 = stock2[:len(stock1)]

keys = ['DMG MORI AG O.N.,Industrial', 'MTU AERO ENGINES NA O.N.,Industrial']
if stock1.shape != stock2.shape:
  new_size = stock1.shape[0]
  stock1 = stock1[:new_size]
  stock2 = stock2[:new_size]
concat_pd = pd.concat([stock1, stock2], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(stock1,stock2)[0][1]

plt.grid()
plt.scatter(stock1, stock2,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('DMG MORI AG O.N.,Industrial'+' and '+'MTU AERO ENGINES NA O.N.,Industrial', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между DMG MORI AG O.N.,Industrial  и MTU AERO ENGINES NA O.N.,Industrial  равен {round(corr_value, 3)}')

Коэфициент корреляции 0.793 свидетельствует о сильной линейной зависимости. что подтверждается графиком.
Активы DMG MORI AG O.N.,Industrial и MTU AERO ENGINES NA O.N.,Industrial из промышленного производственного сектора зависимы между собой.


In [ ]:
import numpy as np

stock2 = tickers[15]['LogReturn'].dropna()
stock1 = tickers[52]['LogReturn'].dropna()
if (len(stock1) > len(stock2)):
  stock1 = stock1[:len(stock2)]
if (len(stock2) > len(stock1)):
  stock2 = stock2[:len(stock1)]

keys = ['DELTICOM AG  NA O.N.,Retail', 'TAKKT AG O.N.,Retail']
if stock1.shape != stock2.shape:
  new_size = stock1.shape[0]
  stock1 = stock1[:new_size]
  stock2 = stock2[:new_size]
concat_pd = pd.concat([stock1, stock2], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(stock1,stock2)[0][1]

plt.grid()
plt.scatter(stock1, stock2,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('DELTICOM AG  NA O.N.,Retail'+' and '+'TAKKT AG O.N.,Retail', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между DELTICOM AG  NA O.N.,Retail  и TAKKT AG O.N.,Retail  равен {round(corr_value, 3)}')

Коэфициент корреляции 0.792 свидетельствует о сильной линейной зависимости. что подтверждается графиком.
Активы DELTICOM AG  NA O.N.,Retail и TAKKT AG O.N.,Retail из сектора розничной торговли зависимы между собой.

Анализ зависимости доходностей у активов из разных производственных секторов:

In [ ]:
import numpy as np

stock2 = tickers[1]['LogReturn'].dropna()
stock1 = tickers[34]['LogReturn'].dropna()
if (len(stock1) > len(stock2)):
  stock1 = stock1[:len(stock2)]
if (len(stock2) > len(stock1)):
  stock2 = stock2[:len(stock1)]

keys = ['3U HOLDING AG,Industrial', 'HEIDELBERGCEMENT AG O.N.,Construction']
if stock1.shape != stock2.shape:
  new_size = stock1.shape[0]
  stock1 = stock1[:new_size]
  stock2 = stock2[:new_size]
concat_pd = pd.concat([stock1, stock2], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(stock1,stock2)[0][1]

plt.grid()
plt.scatter(stock1, stock2,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('3U HOLDING AG,Industrial'+' and '+'HEIDELBERGCEMENT AG O.N.,Construction', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между 3U HOLDING AG,Industrial  и HEIDELBERGCEMENT AG O.N.,Construction  равен {round(corr_value, 3)}')

Коэффициент корреляции близок к нулю. Это говорить об отсутствии зависимость между активами 3U HOLDING AG,Industrial промышленного производства и HEIDELBERGCEMENT AG O.N.,Construction из строительного сектора. Возможно отсутствуют факторы одновременно влияющие на два этих сектора.

In [ ]:
import numpy as np

stock2 = tickers[15]['LogReturn'].dropna()
stock1 = tickers[2]['LogReturn'].dropna()
if (len(stock1) > len(stock2)):
  stock1 = stock1[:len(stock2)]
if (len(stock2) > len(stock1)):
  stock2 = stock2[:len(stock1)]

keys = ['DEX,DELTICOM AG  NA O.N.,Retail', '4SC AG,Pharma']
if stock1.shape != stock2.shape:
  new_size = stock1.shape[0]
  stock1 = stock1[:new_size]
  stock2 = stock2[:new_size]
concat_pd = pd.concat([stock1, stock2], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(stock1,stock2)[0][1]

plt.grid()
plt.scatter(stock1, stock2,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('DEX,DELTICOM AG  NA O.N.,Retail'+' and '+'4SC AG,Pharma', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между DEX,DELTICOM AG  NA O.N.,Retail  и 4SC AG,Pharma  равен {round(corr_value, 3)}')

Коэффициент корреляции близок к нулю, активы DEX,DELTICOM AG  NA O.N.,Retail и 4SC AG,Pharma независимы друг от друга.

In [ ]:
import numpy as np

stock2 = tickers[21]['LogReturn'].dropna()
stock1 = tickers[39]['LogReturn'].dropna()
if (len(stock1) > len(stock2)):
  stock1 = stock1[:len(stock2)]
if (len(stock2) > len(stock1)):
  stock2 = stock2[:len(stock1)]

keys = ['DT.PFANDBRIEFBK AG,Banks', 'LEG IMMOBILIEN SE,Financial Services']
if stock1.shape != stock2.shape:
  new_size = stock1.shape[0]
  stock1 = stock1[:new_size]
  stock2 = stock2[:new_size]
concat_pd = pd.concat([stock1, stock2], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(stock1,stock2)[0][1]

plt.grid()
plt.scatter(stock1, stock2,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('DT.PFANDBRIEFBK AG,Banks'+' and '+'LEG IMMOBILIEN SE,Financial Services', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между DT.PFANDBRIEFBK AG,Banks  и LEG IMMOBILIEN SE,Financial Services  равен {round(corr_value, 3)}')

Коэфициент корреляции 0.555 свидетельствует о наличие линейной зависимости, показанной на графике.
Активы DT.PFANDBRIEFBK AG,Banks и LEG IMMOBILIEN SE,Financial Services из из разных производственных секторов зависимы между собой. Этот факт можно объяснить наличием влияния сфер
друг на друга, общих факторов из вне, так как финансовая сфера и банки тесно связаны друг с другом


In [ ]:
import numpy as np

stock2 = tickers[84]['LogReturn'].dropna()
stock1 = tickers[41]['LogReturn'].dropna()
if (len(stock1) > len(stock2)):
  stock1 = stock1[:len(stock2)]
if (len(stock2) > len(stock1)):
  stock2 = stock2[:len(stock1)]

keys = ['DATRON AG INH.O.N.,Industrial', 'LINDE PLC EO 0,001,Chemicals']
if stock1.shape != stock2.shape:
  new_size = stock1.shape[0]
  stock1 = stock1[:new_size]
  stock2 = stock2[:new_size]
concat_pd = pd.concat([stock1, stock2], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(stock1,stock2)[0][1]

plt.grid()
plt.scatter(stock1, stock2,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('DATRON AG INH.O.N.,Industrial'+' and '+'LINDE PLC EO 0,001,Chemicals', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между DATRON AG INH.O.N.,Industrial  и LINDE PLC EO 0,001,Chemicals  равен {round(corr_value, 3)}')

Активы DATRON AG INH.O.N.,Industrial и LINDE PLC EO 0,001,Chemicals из разных производственных секторов зависимы между собой.

Анализ зависимости объема продаж с доходностями у одного актива

In [ ]:
import numpy as np

log = tickers[58]['LogReturn'].dropna()
volume = tickers[58]['Volume'].dropna()
if (len(log) > len(volume)):
  log = log[:len(volume)]
if (len(volume) > len(log)):
  volume = volume[:len(log)]
keys = ['LogReturn', 'Volume']
if volume.shape != log.shape:
  new_size = volume.shape[0]
  volume = volume[:new_size]
  log = log[:new_size]
concat_pd = pd.concat([volume, log], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(volume,log)[0][1]

plt.grid()
plt.scatter(volume, log,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('LogReturn and Volume', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между доходностью и объёмом продаж актива SINGULUS TECHNOL. EO 1,Industrial равен {round(corr_value, 3)}')

Коэффициент корреляции отрицательный и близок к нулю, что говорит об отсутствие зависимости между доходностями и объемом продаж у актива.


In [ ]:
import numpy as np

log = tickers[15]['LogReturn'].dropna()
volume = tickers[15]['Volume'].dropna()
if (len(log) > len(volume)):
  log = log[:len(volume)]
if (len(volume) > len(log)):
  volume = volume[:len(log)]
keys = ['LogReturn', 'Volume']
if volume.shape != log.shape:
  new_size = volume.shape[0]
  volume = volume[:new_size]
  log = log[:new_size]
concat_pd = pd.concat([volume, log], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(volume,log)[0][1]

plt.grid()
plt.scatter(volume, log,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('LogReturn and Volume', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между доходностью и объёмом продаж актива DELTICOM AG  NA O.N.,Retail равен {round(corr_value, 3)}')

Коэффициент корреляции положительный и близок к нулю, что говорит об отсутствие зависимости между доходностями и объемом продаж у актива.


In [ ]:
import numpy as np

log = tickers[39]['LogReturn'].dropna()
volume = tickers[39]['Volume'].dropna()
if (len(log) > len(volume)):
  log = log[:len(volume)]
if (len(volume) > len(log)):
  volume = volume[:len(log)]
keys = ['LogReturn', 'Volume']
if volume.shape != log.shape:
  new_size = volume.shape[0]
  volume = volume[:new_size]
  log = log[:new_size]
concat_pd = pd.concat([volume, log], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(volume,log)[0][1]

plt.grid()
plt.scatter(volume, log,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('LogReturn and Volume', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между доходностью и объёмом продаж актива LEG IMMOBILIEN SE,Financial Services равен {round(corr_value, 3)}')

In [ ]:
import numpy as np

log = tickers[34]['LogReturn'].dropna()
volume = tickers[34]['Volume'].dropna()
if (len(log) > len(volume)):
  log = log[:len(volume)]
if (len(volume) > len(log)):
  volume = volume[:len(log)]
keys = ['LogReturn', 'Volume']
if volume.shape != log.shape:
  new_size = volume.shape[0]
  volume = volume[:new_size]
  log = log[:new_size]
concat_pd = pd.concat([volume, log], axis=1, sort=False, keys=keys)
corr = concat_pd.corr()
corr_value = np.corrcoef(volume,log)[0][1]

plt.grid()
plt.scatter(volume, log,c = 'blue', edgecolors='black')
plt.xlabel(keys[0])
plt.ylabel(keys[1])
plt.title('LogReturn and Volume', size=15)
plt.show()
some_print(corr)
print(f'Коэффициент корреляции между доходностью и объёмом продаж актива HEIDELBERGCEMENT AG O.N.,Construction равен {round(corr_value, 3)}')

Коэффициенты корреляции отрицательные и близки к нулю, что говорит об отсутствие зависимости между доходностями и объемом продаж у активов.
